In [1]:
from functools import wraps
import cProfile
import pstats
import io



def cProfiler(func):
    """Decorator to profile a function to measure its execution time."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        profiler = cProfile.Profile()
        profiler.enable()
        results = func(*args, **kwargs)
        profiler.disable()
        stream  = io.StringIO()
        stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')
        stats.print_stats()
        return results, stream.getvalue()
    return wrapper

def cprofiler_func(func, *args, **kwargs):
    """Function to profile a function to measure its execution time."""
    profiler = cProfile.Profile()
    profiler.enable()
    results = func(*args, **kwargs)
    profiler.disable()
    stream  = io.StringIO()
    stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')
    stats.print_stats()
    return results, stream.getvalue()


profiler = cProfile.Profile()
profiler.enable()


profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('calls')

In [14]:
import os
import signal
import time

def signal_handler(signum, frame):
    print(f"Signal {signum} received by process {os.getpid()}")
    

# Register a signal handler for SIGUSR1
signal.signal(signal.SIGUSR1, signal_handler)
signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

def child_process():
    print(f"Child process started with PID {os.getpid()}")

    try:
        print("Child process running...")
        time.sleep(2)  # Simulate work
        raise ValueError("Simulated exception in child process")
    except ValueError as e:
        print(f"Child process caught exception: {e}")
    finally:
        print("Child process exiting...")
        os._exit(0)  # Ensure the child process exits cleanly

def child_process2():
    print(f"Child process started with PID {os.getpid()}")
    time.sleep(2)  # Simulate some work
    raise KeyboardInterrupt("Simulated exception in child process")  # Simulate an exception

def parent_process(child_pid):
    print(f"Parent process with PID {os.getpid()}, child PID: {child_pid}")
    try:
        print("Parent process running...")
        time.sleep(5)  # Simulate work
        print("Parent process sending SIGUSR1 to child...")
        os.kill(child_pid, signal.SIGUSR1)  # Send a signal to the child process
    except Exception as e:
        print(f"Parent process caught exception: {e}")
    finally:
        print("Parent process waiting for child to exit...")
        os.wait()  # Wait for the child process to exit
        print("Parent process exiting...")

if __name__ == "__main__":
    pid = os.fork()  # Fork the process

    if pid == 0:
        print("Hi I am a child process with pid", os.getpid(), pid)
        # This is the child process
        child_process2()
    else:
        # This is the parent process
        parent_process(pid)

Hi I am a child process with pid 18133 0
Child process started with PID 18133
Parent process with PID 16447, child PID: 18133
Parent process running...


KeyboardInterrupt: Simulated exception in child process

Signal 30 received by process 18133
Parent process sending SIGUSR1 to child...
Parent process waiting for child to exit...
Signal 2 received by process 16447


In [176]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'CRITICAL'
os.environ['PROXY_URL'] = ''
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd, MonthBegin, BDay
from trade.assets.Stock import Stock
from module_test.raw_code.DataManagers.DataManagers import set_save_bool, get_save_bool, save_to_database, init_query
from module_test.raw_code.DataManagers.DataManagers import *
from trade.helpers.helper import generate_option_tick_new, parse_option_tick, check_all_days_available, check_missing_dates
from module_test.raw_code.DataManagers.Requests import OptionQueryRequestParameter, BulkOptionQueryRequestParameter, create_request_bulk
# from module_test.raw_code.DataManagers.SaveManager import SaveManager
from dbase.database.SQLHelpers import DatabaseAdapter
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
kwargs = {"start": "2023-01-04 00:00:00", "end": "2023-12-29 00:00:00", "tick": "AMZN", "exp": "2024-09-20 00:00:00", "print_info": False, "save_func": "save_to_database", "type_": "bulk", '_requests': []}
# kwargs = {"exp": "2024-01-19", "right": "C", "strike": 480.0, "start": "2023-01-04 00:00:00", "end": "2023-12-29 00:00:00", "tick": "NFLX", "type_": "single", "save_func": "save_to_database", '_requests': []}
# kwargs = {"exp": "2024-01-19", "right": "C", "strike": 130.0, "start": "2023-10-20", "end": "2023-10-25", "tick": "SBUX", "type_": "single", "save_func": "save_to_database"}
kwargs["_requests"] = []
test_bulk = create_request_bulk(**kwargs)

In [172]:
SaveManager.schedule(kwargs)



Scheduled Data Requests will be saved to: /Users/chiemelienwanisobi/cloned_repos/QuantTools/module_test/raw_code/DataManagers/scheduler/requests.jsonl


In [149]:
def check_if_already_saved(request, spot_sm):
    """
    Check if the data has already been saved to the database.
    """
    # Check if the data has already been saved
    db = DatabaseAdapter()
    if isinstance(request,OptionQueryRequestParameter):
        filter_data = init_query(data_request=request, db=db, query_category='single')
        filter_data['Datetime'] = pd.to_datetime(filter_data['datetime'])
        missing_dates = check_missing_dates(filter_data, request.start_date, request.end_date)
        return spot_sm[spot_sm.index.isin(missing_dates)]
    elif isinstance(request,BulkOptionQueryRequestParameter):
        ## Get all strikes to query
        strikes = list(set(spot_sm[['k', 'Right']].itertuples(name = None, index = False)))
        request.strikes = strikes
        filter_data = init_query(data_request=request, db=db, query_category='bulk')
        available = filter_data.set_index(['optiontick', 'datetime']).index
        unavailable = spot_sm.reset_index().set_index(['OptionTick', 'Datetime']).index
        mask = ~unavailable.isin(available)

        ## Filter the data to get the missing data
        missing_data = spot_sm[mask]
        return missing_data
    else:
        raise ValueError("Invalid request type. Must be OptionQueryRequestParameter or BulkOptionQueryRequestParameter.")


In [161]:
def save_to_database(data_request: 'RequestParameter', 
                     print_info: bool = False, 
                     pool: bool = None,
                     **kwargs) -> None:
    """
    Saves the data to the database
    """
    func_start = time.time()
    req_class = data_request.__class__.__name__
    ## This function is using parallel apply to reduce overhead on the current process.
    if pool is None:
        pool = get_pool_enabled()
    print(f"Pool: {pool}", flush=True) if print_info else None
    logger.info(f"Saving data to {data_request.db_name}.{data_request.table_name}")
    print(f"Saving data to {data_request.db_name}.{data_request.table_name}", flush=True) if print_info else None
    
    ## Determine if the data is bulk or not
    if isinstance(data_request, OptionQueryRequestParameter):
        bulk = False
    elif isinstance(data_request, BulkOptionQueryRequestParameter):
        data_request.strike = None
        data_request.right = None
        bulk = True
    else:
        raise ValueError(f"Expected data_request to be of type OptionQueryRequestParameter or BulkOptionQueryRequestParameter, recieved {type(data_request)}")
    
    db = DatabaseAdapter()
    if len(data_request.missing_dates) == 0:
        print("No missing data, skipping save to database", flush=True) if print_info else None
        logger.warning("No missing data, skipping save to database")
        return
    if bulk:
        start_date, end_date  = pd.to_datetime(min(data_request.missing_dates)) - relativedelta(months=1), pd.to_datetime(max(data_request.missing_dates)) + relativedelta(months=1)
    else:
        start_date, end_date = pd.to_datetime(min(data_request.missing_dates)) - relativedelta(months=3), pd.to_datetime(max(data_request.missing_dates)) + relativedelta(months=3)
    logger.info(f"Querying data from {start_date} to {end_date}")
    print(f"Querying data from {start_date} to {end_date} for {req_class}", flush=True) if print_info else None
    
    ## Start by populating initial data from spot_manager
    spot_manager = SpotDataManager(data_request.symbol)
    start = time.time()
    print("Starting to query data", flush=True) if print_info else None
    spot_sm = spot_manager.query_thetadata(start_date, end_date,
                                                   strike=data_request.strike, exp=data_request.exp, 
                                                   right=data_request.right, bulk=bulk, 
                                                   data_request=data_request)
    print("Time taken to query data: ", time.time() - start, flush=True) if print_info else None
    spot_sm = spot_sm.fillna(0)
    spot_sm.drop_duplicates(inplace=True)
    spot_sm = check_if_already_saved(data_request, spot_sm)
    if spot_sm.empty:
        print("All data in Database, returning") if print_info else None
        return 
    return spot_sm

In [180]:
spot_sm = save_to_database(test_bulk, print_info=True, pool=False)



Scheduled Data Requests will be saved to: /Users/chiemelienwanisobi/cloned_repos/QuantTools/module_test/raw_code/DataManagers/scheduler/requests.jsonl
Pool: False
Saving data to securities_master.temp_options_eod_new
Querying data from 2022-12-04 00:00:00 to 2024-01-29 00:00:00 for BulkOptionQueryRequestParameter
Starting to query data
Time taken to query data:  118.7481472492218


In [181]:
spot_sm

,ticker,k,exp_date,Right,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint,OptionTick,Open_interest
Datetime,,,,,,,,,,,,,,,,,
2022-12-12,AMZN,45.0,2024-09-20,C,50.20,50.65,50.20,50.65,2,51,50.70,61,52.40,51.550,51.625893,AMZN20240920C45,0.0
2022-12-13,AMZN,45.0,2024-09-20,C,0.00,0.00,0.00,0.00,0,50,52.55,50,54.55,53.550,53.550000,AMZN20240920C45,0.0
2022-12-14,AMZN,45.0,2024-09-20,C,0.00,0.00,0.00,0.00,0,50,51.55,50,53.80,52.675,52.675000,AMZN20240920C45,0.0
2022-12-15,AMZN,45.0,2024-09-20,C,49.91,49.91,49.91,49.91,1,11,45.00,28,54.50,49.750,51.820513,AMZN20240920C45,0.0
2022-12-16,AMZN,45.0,2024-09-20,C,48.61,48.73,48.61,48.73,2,50,48.10,50,50.25,49.175,49.175000,AMZN20240920C45,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-29,AMZN,230.0,2024-09-20,P,0.00,0.00,0.00,0.00,0,68,68.30,67,69.10,68.700,68.697037,AMZN20240920P230,0.0
2024-01-26,AMZN,235.0,2024-09-20,C,1.10,1.12,1.09,1.12,37,117,1.09,119,1.14,1.115,1.115212,AMZN20240920C235,0.0
2024-01-29,AMZN,235.0,2024-09-20,C,1.15,1.28,1.12,1.28,46,120,1.27,103,1.31,1.290,1.288475,AMZN20240920C235,37.0


In [117]:
data = init_query(data_request = test_bulk, db = DatabaseAdapter(), query_category='single')
data.set_index('datetime', inplace=True)

In [122]:
data['Datetime'] = data.index
check_all_days_available(data, test_bulk.start_date, test_bulk.end_date)

True

In [120]:
data
test_bulk.start_date, test_bulk.end_date

,open,high,low,close,volume,bid_size,closebid,ask_size,closeask,strike,...,ask_binomial_vanna,ask_binomial_volga,bid_binomial_delta,bid_binomial_gamma,bid_binomial_vega,bid_binomial_rho,bid_binomial_theta,bid_binomial_vanna,bid_binomial_volga,Datetime
datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-20,0.08,0.08,0.07,0.07,12,1,0.06,22,0.07,130.0,...,0.126648,12.802562,0.013804,0.002698,0.016602,0.003089,-0.002597,0.116286,12.030932,2023-10-20
2023-10-23,0.06,0.06,0.06,0.06,1,3,0.03,11,0.10,130.0,...,0.148433,14.000071,0.008000,0.001825,0.010176,0.001743,-0.001529,0.078108,8.692789,2023-10-23
2023-10-24,0.06,0.06,0.05,0.05,3,90,0.04,99,0.06,130.0,...,0.115663,11.740375,0.010436,0.002234,0.012810,0.002252,-0.002015,0.095009,10.127514,2023-10-24
2023-10-25,0.05,0.05,0.05,0.05,4,289,0.03,280,0.06,130.0,...,0.111993,11.249215,0.007970,0.001811,0.010004,0.001693,-0.001564,0.076504,8.403964,2023-10-25


In [127]:
def is_single_already_in_db(req):
    """
    Check if the data has already been saved to the database.
    """
    # Check if the data has already been saved
    db = DatabaseAdapter()
    if isinstance(req,OptionQueryRequestParameter):
        pass
    elif isinstance(req, dict):
        req["_requests"] = []
        req = create_request_bulk(**req)
    else:
        raise ValueError(f"Invalid request type. Must be OptionQueryRequestParameter or Dict. Received {type(req)}")
    
    database_data = init_query(data_request=req, db=db, query_category='single')
    database_data['Datetime'] = pd.to_datetime(database_data['datetime'])
    bool_check = check_all_days_available(database_data, req.start_date, req.end_date)
    return bool_check
    

In [128]:
is_single_already_in_db(kwargs)

True

## TEST DATA MANAGER

In [4]:
# manager = OptionDataManager('AAPL', '2025-09-19', 'C', 225.0) ## EOD
manager = OptionDataManager('AAPL', '2025-09-19', 'P', 290.0) ## Intra
spot_manager = manager.spot_manager

Saving to cache from db


In [5]:
request_ = manager.get_at_time('2024-12-30', 'spot', return_price=False, model = 'binomial', extra_cols = ['ask'])
request_

,Open,High,Low,Close,Midpoint,Volume,Openinterest,Closeask
Datetime,,,,,,,,
2024-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
pprint(BulkOptionDataManager._REQUESTS)

[]


In [86]:
exp = '2024-03-15'
start = '2023-06-02'
end = '2024-06-02'
BulkOptionDataManager.one_off_save(
    start = pd.to_datetime(start),
    end = pd.to_datetime(end),
    tick = 'AAPL',
    exp = exp,
    print_info = True
)

Printing info
[SaveManager] Enqueueing save request for AAPL on <module_test.raw_code.DataManagers.DataManagers.BulkOptionQueryRequestParameter object at 0x13f38ba50>


Saving data to securities_master.temp_options_eod_new
Querying data from 2023-03-02 00:00:00 to 2024-09-02 00:00:00
Starting to save data to database
Size of spot data:  (36596, 22)
Size of spot data after filtering:  (25684, 23)
Calculating Vols


In [9]:
req = SaveManager.status()[ 'current_requests']['Thread-5 (_worker)']

In [91]:
pprint(SaveManager.status())

{'active_threads': 4,
 'current_requests': {'SaveWorker-0': <module_test.raw_code.DataManagers.DataManagers.BulkOptionQueryRequestParameter object at 0x13f38ba50>},
 'max_queue_size': 100,
 'num_failed_requests': 0,
 'num_finished_requests': 0,
 'pending_tasks': 0,
 'total_threads': 4}


In [81]:
SaveManager.restart_workers()

[SaveManager] All save workers have been killed.
[SaveManager] Restarted all save workers.


In [12]:
req = SaveManager._failed_requests[0]
req.error


KeyError('UPPER_BOUND_MONEYNES')

In [ ]:
data = retrieve_bulk_eod(
    symbol = 'NFLX',
    exp = req.exp,
    start_date= req.start_date,
    end_date = req.end_date,
    end_time = '16:00',
    right = None
)
data

,Root,Strike,Expiration,Right,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,,,,,
2023-06-02 16:00:00,NFLX,5.0,2024-03-15,C,0.00,0.00,0.00,0.00,0,35,392.20,25,400.15,396.175,395.512500
2023-06-05 16:00:00,NFLX,5.0,2024-03-15,C,0.00,0.00,0.00,0.00,0,25,394.30,15,403.25,398.775,397.656250
2023-06-06 16:00:00,NFLX,5.0,2024-03-15,C,396.74,396.74,395.22,395.22,2,25,390.30,15,399.00,394.650,393.562500
2023-06-07 16:00:00,NFLX,5.0,2024-03-15,C,0.00,0.00,0.00,0.00,0,25,391.75,15,399.50,395.625,394.656250
2023-06-08 16:00:00,NFLX,5.0,2024-03-15,C,0.00,0.00,0.00,0.00,0,15,400.45,10,405.85,403.150,402.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11 16:00:00,NFLX,960.0,2024-03-15,P,0.00,0.00,0.00,0.00,0,25,356.85,25,361.30,359.075,359.075000
2024-03-12 16:00:00,NFLX,960.0,2024-03-15,P,0.00,0.00,0.00,0.00,0,13,348.10,25,350.85,349.475,349.909211
2024-03-13 16:00:00,NFLX,960.0,2024-03-15,P,0.00,0.00,0.00,0.00,0,6,349.50,25,352.70,351.100,352.080645


# BEGIN TESTS ON RISKMANAGER TIME PERFORMANCE

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from EventDriven.riskmanager import (RiskManager, 
                                     OrderPicker)
from EventDriven.riskmanager.utils import (
                                    produce_order_candidates, 
                                     populate_cache, 
                                     clear_cache, 
                                     return_closePrice,
                                     get_cache,
                                     close_cache,
                                     oi_cache,
                                     spot_cache,
                                     
)
from trade.helpers.threads import runThreads
from trade.helpers.helper import generate_option_tick_new, parse_option_tick, check_all_days_available
from dbase.DataAPI.ThetaData import retrieve_eod_ohlc, retrieve_openInterest
from dbase.utils import default_timestamp, add_eod_timestamp
from functools import partial
def _retrieve_openInterest(*args, **kwargs):
    try:
        return retrieve_openInterest(*args, **kwargs)
    except Exception as e:
        return None
produce_order_candidates_cprofiler = cProfiler(produce_order_candidates)

Using Old DataManager


## GET ORDER Improvements

### PRODUCE_ORDER_CANDIDATES

- Used HOLIDAY_SETS in is_USHoliday to optimize time

In [8]:
clear_cache()

In [9]:
order = {'type': 'naked', 
         'specifics': [{'direction': 'long', 'rel_strike': 0.85, 'dte': 300, 'moneyness_width': 0.15}, 
                                        {'direction': 'short', 'rel_strike': 0.6, 'dte': 300, 'moneyness_width': 0.15}], 
                                        'name': 'vertical_spread'}
candidates, stats = produce_order_candidates_cprofiler(order,
                         'NVDA',
                         '2023-06-02',
                         'C',
                         False)

Saving to cache from db


In [10]:
print(stats)

         4447279 function calls (4420341 primitive calls) in 22.537 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   22.537   22.537 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/riskmanager/utils.py:655(produce_order_candidates)
        2    0.001    0.000   22.537   11.269 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/riskmanager/utils.py:519(chain_details)
        1    0.000    0.000   18.774   18.774 /Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/assets/Stock.py:85(__init__)
        1    0.000    0.000   16.689   16.689 /Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/assets/Stock.py:216(init_rfrate_ts)
        1    0.000    0.000   16.689   16.689 /Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/assets/rates.py:178(get_risk_free_rate_helper)
        1    0.002    0.002   16.427   16.427 /Users/chiemelienwanisobi/cloned_repos/QuantTool

In [11]:
start  = '2023-06-02'
end = pd.to_datetime(start) + relativedelta(years=+1)
# populate_cache(order_candidates=candidates,
#                start_date=start,
#                end_date=end,
#                target_date='2024-03-13',)

### Populate Cache Improvements

#### Base Test

In [42]:
candidates

##Test 1: Using runThreads as before

@cProfiler
def runThreads_method():
    full_data = pd.DataFrame()
    for direction in candidates:
        for data in candidates[direction]:


            full_data = pd.concat([full_data, data], axis=0)

    full_data.index.name = 'Date'
    full_data.columns.name = ''
    full_data['start_date'] = start
    full_data['end_date'] = end
    full_data.reset_index( inplace=True)
    OrderedList = full_data[['ticker', 'end_date', 'Expiration', 'Right', 'start_date', 'Strike', ]].T.to_numpy()
    tickOrderedList = full_data[['ticker', 'Right', 'Expiration', 'Strike', ]].T.to_numpy()
    eod_results = (runThreads(retrieve_eod_ohlc, OrderedList, 'map', block=False))
    oi_results = (runThreads(_retrieve_openInterest, OrderedList, 'map' , block=False))
    tick_results = (runThreads(generate_option_tick_new, tickOrderedList, 'map' , block=False))

    eod_results = list(eod_results)
    oi_results = list(oi_results)
    tick_results = list(tick_results)

    return eod_results, oi_results, tick_results

# pack, stats = runThreads_method()

#### Improvement. Hybrid of runThreads method & Querying database

In [12]:
def assemble_bulk_data_request(self, start: str | datetime, 
                       end: str | datetime,
                       interval: str = '1d',
                       type_: str = 'spot',
                       strikes_right: List[Tuple] = [],
                       model: str = 'bs',
                       extra_cols: list = []) :
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    ivl_str, ivl_int = extract_numeric_value(interval)
    greek_names = self.greek_names
    _extra_cols = handle_extra_cols(extra_cols, type_, model)
    greek_cols = build_name_format('greek', model, extra_cols, self.default_fill)
    vol_cols = build_name_format('vol', model, extra_cols, self.default_fill)


    ## Enforce the interval
    enforce_interval(ivl_str)

    ## Assert inputs
    enforce_inputs(type_, model)

    ## Determine aggregation
    agg, database, table = determine_table_agg(ivl_str, type_, greek_names)
    input_params = getattr(self, agg)

    ## Determine the requested columns
    requested_col = determine_requested_columns(self.default_fill, type_, model, greek_names)

    data_request = BulkOptionQueryRequestParameter(table_name=table,
                                                        db_name=database, 
                                                        start_date=start, 
                                                        end_date=end, 
                                                        ticker=self.symbol, 
                                                        exp=self.exp, 
                                                        strikes=strikes_right)

    ## Set the parameters for the request to avoid having too many attributes
    data_request.symbol = self.symbol
    data_request.interval= interval
    data_request.type_ = type_
    data_request.input_params = input_params
    data_request.model = model
    data_request.ivl_str = ivl_str
    data_request.ivl_int = ivl_int
    data_request.default_fill = self.default_fill
    data_request.agg = agg
    data_request.requested_col = requested_col + _extra_cols + ['optiontick']
    data_request.iv_cols = vol_cols
    data_request.greek_cols = greek_cols
    data_request.col_kwargs = col_kwargs = {'underlier_price': 's0', 
            'expiration': 'exp_date', 
            'strike': 'k', 
            'right': 'right', 
            'rf_rate': 'r', 
            'dividend': 'y',
            'put/call': 'right',
            'datetime': 'datetime',}
    return data_request

In [13]:

def check_all_days_available(x, _start, _end):
    # print(x)
    date_range = bus_range(_start, _end, freq = '1B')
    dates_available = x.Datetime
    missing_dates_second_check = [x for x in date_range if x not in pd.DatetimeIndex(dates_available)]
    return all(x in pd.DatetimeIndex(dates_available) for x in date_range)

def check_missing_dates(x, _start, _end):
    # print(x)
    date_range = bus_range(_start, _end, freq = '1B')
    dates_available = x.Datetime
    missing_dates_second_check = [x for x in date_range if x not in pd.DatetimeIndex(dates_available)]
    return missing_dates_second_check

def update_caches(x):
    global oi_cache, close_cache, oi_cache, spot_cache
    key = f"{x.Optiontick.unique()[0]}"
    x = x.set_index('Datetime')
    close_cache[key] = x
    oi_cache[key] = x['Openinterest'].to_frame(name = 'Open_interest')
    pass




def update_cache_with_missing_ticks(parsed_opts: pd.DataFrame, ) -> None:
    """
    Updates the cache with missing ticks by retrieving EOD data and open interest data.

    Args:
        parsed_opts (pd.DataFrame): DataFrame containing parsed option ticks with start and end dates and key meta data.

    Returns:
        None
    """
    global oi_cache, close_cache, oi_cache, spot_cache
    OrderedList = parsed_opts[['ticker', 'end_date', 'exp_date', 'put_call', 'start_date', 'strike', ]].T.to_numpy()
    tickOrderedList = parsed_opts[['ticker', 'put_call', 'exp_date', 'strike', ]].T.to_numpy()
    eod_results = (runThreads(retrieve_eod_ohlc, OrderedList, 'map', block=True))
    oi_results = (runThreads(_retrieve_openInterest, OrderedList, 'map' , block=True))
    tick_results = (runThreads(generate_option_tick_new, tickOrderedList, 'map' , block=True))

    eod_results = list(eod_results)
    oi_results = list(oi_results)
    tick_results = list(tick_results)

    for oi, eod, tick in zip(oi_results, eod_results, tick_results):
        cache_key = f"{tick}"
        eod.index = default_timestamp(eod.index)
        eod['Optiontick'] = tick
        close_cache[cache_key] = eod

        ## OI Formatting for consistency
        oi.set_index('Datetime', inplace=True)
        oi.index = default_timestamp(oi.index)
        oi_cache[cache_key] = oi

    return



In [14]:

def return_closePrice(id: str, 
                      date: str) -> float:
    """
    returns the close price of the option contract
    id: str: id of the option contract, corresponding to cache keys.
        ps: Use spot_cache.keys() to get the keys
    date: str: date to get the close price for

    returns:
    float: close price of the option contract
    
    """
    global close_cache, spot_cache, oi_cache
    cache_key = f"{id}"  ## Close Uses only the id, not the date
    close_data = close_cache[cache_key]
    if close_data is None:
        return None
    close_data = close_data[~close_data.index.duplicated(keep = 'first')]
    if date not in close_data.index:
        ## If the date is not in the close data, we remove that key from the cache
        ## There's no way to resolve this, so we remove the key from the cache
        try:
            close_cache.pop(cache_key)
            oi_cache.pop(cache_key)
        except KeyError:
            pass
        return None
    close = close_data['Midpoint'][date]
    return close


In [15]:
def organize_data_for_query(missing_list: list,
                            incomplete_dict: dict,
                            data_request: 'DataManagers.Request') -> pd.DataFrame:
    """
    Organizes the data for the query by parsing the option ticks and adding start and end dates.
    
    Args:
        missing_list (list): List of missing option ticks. These are ticks that are not in the database at all.
        incomplete_dict (dict): Dictionary of incomplete option ticks. These are ticks that are in the database but not complete.
        data_request (BulkOptionQueryRequestParameter): The data request object containing start and end dates.
        
    Returns:
    pd.DataFrame: A DataFrame containing the parsed option ticks with start and end dates.
    """
    parsed_opts = pd.DataFrame()
    ## First populate with the ticks completely missing.
    parsed_opts = pd.DataFrame([parse_option_tick(x) for x in missing_list])
    parsed_opts[['start_date', 'end_date']] = data_request.start_date, data_request.end_date

    ## Next populate with the ticks that are incomplete
    for opt, _list in incomplete_dict.items():
        if len(_list) == 0:
            continue
        opt_meta = parse_option_tick(opt)
        opt_meta['start_date'] = min(_list)
        opt_meta['end_date'] = data_request.end_date
        parsed_opts = pd.concat([parsed_opts, pd.DataFrame(opt_meta, index = [0])], axis=0)
    return parsed_opts

def drop_cache_duplicates():
    """
    Drops duplicates from the cache.
    """
    global close_cache, oi_cache
    for key in close_cache.keys():
        close_cache[key] = close_cache[key][~close_cache[key].index.duplicated(keep = 'first')]
    for key in oi_cache.keys():
        oi_cache[key] = oi_cache[key][~oi_cache[key].index.duplicated(keep = 'first')]

    return

def merge_incomplete_data_in_cache(
    incomplete_dict: dict,
    pre_processed_data: pd.DataFrame,
):
    global close_cache, oi_cache, spot_cache
    ## Now we have updated cache, since incomplete date updates cache with the missing dates, we have to add the data we already have
    for tick, _list in incomplete_dict.items():
        if len(_list) == 0:
            continue
        tick_data = pre_processed_data[pre_processed_data.Optiontick == tick]
        tick_data = tick_data.set_index('Datetime')
        close_cache[tick] = pd.concat([close_cache[tick], tick_data], axis=0).sort_index()
        oi_data = tick_data['Openinterest'].to_frame(name = 'Open_interest')
        oi_cache[tick] = pd.concat([oi_cache[tick], oi_data]).sort_index()

def update_spot_cache(opttick: list, target_date: str|datetime) -> None:
    """
    Updates the spot cache with the close price for the given option tick and target date.
    Args:
        opttick (list): List of option ticks.
        target_date (str|datetime): Target date to get the close price for.
    Returns:
        None
    """
    global spot_cache
    spot_results = runThreads(return_closePrice, [opttick, [target_date]*len(opttick)], 'map')
    for tick, spot in zip(opttick, spot_results):
        cache_key = f"{tick}_{target_date}"
        if spot is None:
            continue ## If spot is None, we don't update the cache
        spot_cache[cache_key] = spot

In [31]:
def populate_cache_v2(
        start,
        end,
        candidates,
        target_date,
):
    """
    populates the cache with the necessary data for the order candidates
    This version will improve on the previous one by using the new BulkOptionDataManager
    The goal is to make use of our database to speed up queries where possible

    params:
    candidates: dict: dictionary containing the order candidates
        example: {'type': 'naked',
                    'specifics': [{'direction': 'long',
                    'rel_strike': .900,
                    'dte': 365,
                    'moneyness_width': 0.15},
                    {'direction': 'short',
                    'rel_strike': .80,
                    'dte': 365,
                    'moneyness_width': 0.15}],

                    'name': 'vertical_spread'}
    start: str: date to populate the cache for
    end: str: date to populate the cache for
    target_date: str: date to populate the cache for

    returns:
    str|None: returns 'holiday' if the date is a holiday, 'theta_data_error' if there is an error in the theta data, None otherwise
    """
    
    print(f"Looks like our young fellow is targetting: {target_date}")
    global oi_cache, close_cache, oi_cache, spot_cache
    start, end = pd.to_datetime(start), pd.to_datetime(end)
    full_data = pd.DataFrame()
    for direction in candidates:
        for data in candidates[direction]:
            if isinstance(data, str) and data =='theta_data_error':
                return 'theta_data_error'
            if pd.to_datetime(target_date).weekday() >= 5:
                return 'weekend'
            full_data = pd.concat([full_data, data], axis=0)

    full_data.index.name = 'Date'
    full_data.columns.name = ''
    full_data['start_date'] = start
    full_data['end_date'] = end
    full_data.reset_index(inplace=True)
    tick = full_data.ticker.unique()[0]
    print(tick)
    exp = full_data.Expiration.unique()[0]
    strikes_right = list(full_data[['Strike', 'Right']].itertuples(name=None, index=False))

    ## Let's start with getting the requested data from database
    # manager = BulkOptionDataManager(symbol='AAPL', exp='2025-09-19')
    manager = BulkOptionDataManager(symbol=tick, exp=exp)
    print(f"Generating Data for {manager.symbol} {manager.exp}")
    data_request = assemble_bulk_data_request(
        self = manager,
         start = start,
        end = end,
        type_ = 'spot',
        strikes_right = strikes_right,
        # strikes_right= [(225.0, 'C'), (280.0, 'P'), (250.0, 'C'), (290.0, 'P'), (270.0, 'C'), (270.0, 'P')],

    )
    # ## Second: we query our database to see what data we have
    init_query(data_request = data_request, query_category = 'bulk')

    # ## Third: we pre_process the data request to see if it is complete
    BulkOptionDataManager.pre_process_data(data_request = data_request) 
    BulkOptionDataManager.one_off_save(
        start=start,
        end=end,
        tick=tick,
        exp=exp
    ) ## We shouldn't keep going to thetadata, that takes time. Submit a process. Don't worry it runs on a new process.
    ## Wouldn't affect current procedures


    is_complete = data_request.pre_process['is_complete']
    pre_processed_data = data_request.pre_processed_data.reset_index()
    opttick = data_request.opttick
    return data_request
    print(f"Data Is_complete bool: {is_complete}")

    ## If complete, Fantastic! We re done, now update cache and get out
    if is_complete:
        pre_processed_data.groupby('Optiontick').apply(update_caches)

    ## If NOT complete, do not fret. We'll simply run our process for incomplete/missing ticks
    else:
        ## We first check for the requested ticks. Which one is not in database at all?
        missing_opttick = [x for x in data_request.opttick if x not in pre_processed_data.Optiontick.unique()]
        
        ## Next we check to see if the requested opttick data is COMPELETE. 
        ## If incomplete, we perform runthreads
        check_partial = partial(check_all_days_available, _start = data_request.start_date, _end = data_request.end_date)
        opttick_complete = pre_processed_data.groupby('Optiontick').apply(check_partial)
        incomplete_ticks = opttick_complete[opttick_complete==False].index.tolist()
        incomplete_dict = pre_processed_data.groupby('Optiontick').apply(check_missing_dates, _start = data_request.start_date, _end = data_request.end_date)
        if incomplete_dict.empty:
            incomplete_dict = {}
        else:
            incomplete_dict = incomplete_dict.to_dict()
        ## Before we perform run Threads, it is important we update cache with the Optticks that are COMPLETE
        available = opttick_complete[opttick_complete==True].index
        pre_processed_data[pre_processed_data.Optiontick.isin(available)].groupby('Optiontick').apply(update_caches)

        ## Produce the dataframe that stores names to update the cache
        to_update_cache_data = organize_data_for_query(
            missing_list=missing_opttick,
            incomplete_dict=incomplete_dict,
            data_request=data_request
        )



        # Now my dear friends, we update cache of unavailable ticks
        pack = update_cache_with_missing_ticks(parsed_opts = to_update_cache_data)

        ## Merge the data we have in cache with the data we just retrieved for the incomplete ticks
        merge_incomplete_data_in_cache(incomplete_dict = incomplete_dict, pre_processed_data = pre_processed_data)
        drop_cache_duplicates()
        print("I'm proud of you, we are finally done")

    print("Actually! We are not done yet. We need to get the spot prices for the requested date")
    
    ## Now we update the spot cache
    update_spot_cache(opttick = opttick, target_date = target_date)

    print("Now, my dear friend, we are done")
    return pack
    



In [36]:
# start = '2024-12-03'
# end = '2025-01-05'
start  = '2023-06-02'
end = pd.to_datetime(start) + relativedelta(years=+1)
to_update_cache_data_return = populate_cache_v2(
    start=start,
    end=end,
    candidates=candidates,
    target_date=pd.to_datetime(start) + relativedelta(weeks=+1)
)
to_update_cache_data_return

Looks like our young fellow is targetting: 2023-06-09 00:00:00
NVDA
Generating Data for NVDA 2024-03-15 00:00:00
[SaveManager] Enqueueing save request for NVDA on {'start': Timestamp('2023-06-02 00:00:00'), 'end': Timestamp('2024-06-02 00:00:00'), 'tick': 'NVDA', 'exp': Timestamp('2024-03-15 00:00:00'), 'print_info': False, 'save_func': functools.partial(<function save_to_database at 0x1453094e0>, print_info=False, pool=False), '_requests': <ListProxy object, typeid 'list' at 0x14bd4f9d0>}


In [42]:
print(to_update_cache_data_return.query)

SELECT *
        FROM securities_master.temp_options_eod_new
        WHERE OPTIONTICK in ('NVDA20240315C395', 'NVDA20240315C400', 'NVDA20240315C405', 'NVDA20240315C410', 'NVDA20240315C415', 'NVDA20240315C420', 'NVDA20240315C425', 'NVDA20240315C430', 'NVDA20240315C435', 'NVDA20240315C440', 'NVDA20240315C445', 'NVDA20240315C450', 'NVDA20240315C455', 'NVDA20240315C460', 'NVDA20240315C465', 'NVDA20240315C470', 'NVDA20240315C475', 'NVDA20240315C480', 'NVDA20240315C485', 'NVDA20240315C490', 'NVDA20240315C495', 'NVDA20240315C500', 'NVDA20240315C505', 'NVDA20240315C510', 'NVDA20240315C515', 'NVDA20240315C520', 'NVDA20240315C525', 'NVDA20240315C530', 'NVDA20240315C535', 'NVDA20240315C540', 'NVDA20240315C545', 'NVDA20240315C550', 'NVDA20240315C555', 'NVDA20240315C560', 'NVDA20240315C525', 'NVDA20240315C530', 'NVDA20240315C535', 'NVDA20240315C540', 'NVDA20240315C545', 'NVDA20240315C550', 'NVDA20240315C555', 'NVDA20240315C560', 'NVDA20240315C565', 'NVDA20240315C570', 'NVDA20240315C575', 'NVDA20240

In [29]:
to_update_cache_data_return.groupby('Optiontick').apply(check_all_days_available, _start = start, _end = end)

Optiontick
AAPL20250919C225     True
AAPL20250919C250    False
AAPL20250919C270     True
AAPL20250919P270     True
AAPL20250919P290     True
dtype: bool

In [34]:
get_cache('close').keys()

dict_keys(['NVDA20240315C395', 'NVDA20240315C400', 'NVDA20240315C405', 'NVDA20240315C410', 'NVDA20240315C415', 'NVDA20240315C420', 'NVDA20240315C425', 'NVDA20240315C430', 'NVDA20240315C435', 'NVDA20240315C440', 'NVDA20240315C445', 'NVDA20240315C450', 'NVDA20240315C455', 'NVDA20240315C460', 'NVDA20240315C465', 'NVDA20240315C470', 'NVDA20240315C475', 'NVDA20240315C480', 'NVDA20240315C485', 'NVDA20240315C490', 'NVDA20240315C495', 'NVDA20240315C500', 'NVDA20240315C505', 'NVDA20240315C510', 'NVDA20240315C515', 'NVDA20240315C520', 'NVDA20240315C525', 'NVDA20240315C530', 'NVDA20240315C535', 'NVDA20240315C540', 'NVDA20240315C545', 'NVDA20240315C550', 'NVDA20240315C555', 'NVDA20240315C560', 'NVDA20240315C565', 'NVDA20240315C570', 'NVDA20240315C575', 'NVDA20240315C580', 'NVDA20240315C590', 'NVDA20240315C600', 'NVDA20240315C610', 'NVDA20240315C620', 'NVDA20240315C630', 'NVDA20240315C640', 'NVDA20240315C650', 'NVDA20240315C660', 'NVDA20240315C670', 'NVDA20240315C680', 'NVDA20240315C690', 'NVDA2024

In [56]:
# db = DatabaseAdapter()

# req = _SaveManager._finished_requests[0]
# db.save_to_database(
#     req.saved_to_db_data,
#     req.db_name,
#     req.table_name,
# )
req.saved_to_db_data['moneyness'] = req.saved_to_db_data['underlier_price'] / req.saved_to_db_data['strike']
req.saved_to_db_data['underlier_price'].unique()

array([26.88100052, 27.03700066, 27.57900047, 27.16900063, 26.49500084,
       26.46299934, 26.75799942, 27.00200081, 27.66699982, 27.93099976,
       27.10400009, 27.11899948, 27.04199982, 26.24099922, 26.95599937,
       27.22599983, 27.74900055, 28.90999985, 28.20999908, 27.80200005,
       27.56200027, 28.68000031, 29.15099907, 28.57099915, 28.88500023,
       28.57799911, 28.34000015, 28.95299911, 29.21299934, 30.1779995 ,
       31.6779995 , 31.26399994, 31.1760006 , 30.68799973, 30.53800011,
       37.47499847, 37.97999954, 38.94599915, 40.11100006, 37.83399963,
       39.77000046, 39.32699966, 39.17100143, 38.65399933, 38.50999832,
       38.77000046, 39.48199844, 41.02199936, 42.99700165, 42.65299988,
       42.69200134, 43.80799866, 43.04499817, 43.02500153, 42.20899963,
       40.63199997, 41.87599945, 41.11700058, 40.8219986 , 42.30199814,
       42.4129982 , 42.31700134, 42.10300064, 42.50299835, 42.18000031,
       42.40499878, 43.90200043, 44.30899811, 44.61199951, 44.26

In [46]:
close_cache.keys()

dict_keys(['NVDA20240315C395', 'NVDA20240315C400', 'NVDA20240315C405', 'NVDA20240315C410', 'NVDA20240315C415', 'NVDA20240315C420', 'NVDA20240315C425', 'NVDA20240315C430', 'NVDA20240315C435', 'NVDA20240315C440', 'NVDA20240315C445', 'NVDA20240315C450', 'NVDA20240315C455', 'NVDA20240315C460', 'NVDA20240315C465', 'NVDA20240315C470', 'NVDA20240315C475', 'NVDA20240315C480', 'NVDA20240315C485', 'NVDA20240315C490', 'NVDA20240315C495', 'NVDA20240315C500', 'NVDA20240315C505', 'NVDA20240315C510', 'NVDA20240315C515', 'NVDA20240315C520', 'NVDA20240315C525', 'NVDA20240315C530', 'NVDA20240315C535', 'NVDA20240315C540', 'NVDA20240315C545', 'NVDA20240315C550', 'NVDA20240315C555', 'NVDA20240315C560', 'NVDA20240315C565', 'NVDA20240315C570', 'NVDA20240315C575', 'NVDA20240315C580', 'NVDA20240315C590', 'NVDA20240315C600', 'NVDA20240315C610', 'NVDA20240315C620', 'NVDA20240315C630', 'NVDA20240315C640', 'NVDA20240315C650', 'NVDA20240315C660', 'NVDA20240315C670', 'NVDA20240315C680', 'NVDA20240315C690', 'NVDA2024

In [44]:
retrieve_eod_ohlc(
    symbol = 'NVDA',
    end_date='2024-03-15',
    exp='2024-03-15',
    right='C',
    start_date='2023-06-02',
    strike=395.0,
    print_url=True
    
)

http://127.0.0.1:25510/v2/hist/option/eod?end_date=20240315&root=NVDA&use_csv=true&exp=20240315&right=C&start_date=20230602&strike=395000


,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-06-02 16:00:00,75.55,75.55,75.55,75.55,1,23,72.85,44,76.00,74.425,74.918657
2023-06-05 16:00:00,71.30,72.02,71.30,72.02,2,80,70.35,111,72.30,71.325,71.483246
2023-06-06 16:00:00,69.00,69.89,65.65,65.65,141,22,66.20,75,68.35,67.275,67.862371
2023-06-07 16:00:00,65.80,65.80,60.94,60.94,19,9,58.70,38,60.85,59.775,60.438298
2023-06-08 16:00:00,62.93,65.45,62.93,65.45,15,8,64.75,148,66.80,65.775,66.694872
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-11 16:00:00,0.00,0.00,0.00,0.00,0,1,462.50,1,464.50,463.500,463.500000
2024-03-12 16:00:00,506.12,506.19,506.12,506.19,4,1,522.35,1,527.70,525.025,525.025000
2024-03-13 16:00:00,496.67,498.27,496.67,498.27,2,5,512.50,1,517.50,515.000,513.333333


In [236]:
candidates

{'long': [Right      Expiration  DTE  Strike        Spot         q        r  \
  build_date                                                          
  2023-06-02 2024-03-15  286   395.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   400.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   405.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   410.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   415.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   420.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   425.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   430.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   435.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   440.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   445.0  393.269997  0.000402  0.05215   
  2023-06-02 2024-03-15  286   450.0  393.269997  0.000402  0.05215  

In [233]:
# pd.concat([oi_cache['AAPL20250919C250'], to_update_cache_data_return])
close_cache['AAPL20250919C250']
spot_cache

{'AAPL20250919C225_2024-12-23': 45.775000000000006,
 'AAPL20250919P280_2024-12-23': 30.5,
 'AAPL20250919C250_2024-12-23': 28.475,
 'AAPL20250919P290_2024-12-23': 37.7,
 'AAPL20250919C270_2024-12-23': 17.0,
 'AAPL20250919P270_2024-12-23': 24.4}

In [67]:
full_data = organize_data_for_query(query_ticks=query_ticks, data_request=req)
full_data

,ticker,put_call,exp_date,strike,start_date,end_date
0,AAPL,P,2025-09-19,280.0,2024-12-03,2025-01-05


#### Other Tests with Data Managers

In [ ]:
## Test 2, using a list of option data managers
def get_timeseries(opm, start, end):
    return opm.get_timeseries(
        start=start,
        end=end,
        interval='1d',
        type_='spot',
        model='bs',
    )

full_data = pd.DataFrame()
for direction in candidates:
    for data in candidates[direction]:
        full_data = pd.concat([full_data, data], axis=0)

full_data.index.name = 'Date'
full_data.columns.name = ''
full_data['start_date'] = start
full_data['end_date'] = end
full_data['Strike'] = full_data.Strike.astype(float)
init_OrderedList = full_data[['ticker', 'Expiration', 'Right', 'Strike']].T.to_numpy()

## Create data manager
opm_list = runThreads(OptionDataManager, init_OrderedList)
opttick = [x.opttick for x in opm_list]

## Ordered list for runThreads
ts_OrderedList = list(full_data[['start_date', 'end_date']].T.to_numpy())
ts_OrderedList.insert(0, opm_list)

## Get Data init
data = (runThreads(get_timeseries, ts_OrderedList, 'map' , block=True))
data

In [239]:
## Test 3, Using BulkDataManager

full_data = pd.DataFrame()
for direction in candidates:
    for data in candidates[direction]:
        full_data = pd.concat([full_data, data], axis=0)

full_data.index.name = 'Date'
full_data.columns.name = ''
full_data['start_date'] = start
full_data['end_date'] = end
full_data['Strike'] = full_data.Strike.astype(float)


bulk_manager = BulkOptionDataManager(full_data.ticker.unique()[0],
                                     full_data.Expiration.unique()[0],
                                     )


timeseries = bulk_manager.get_timeseries(
        start=start,
        end=end,
        interval='1d',
        type_='spot',
        model='bs',
        strikes_right=list(full_data[['Strike', 'Right']].itertuples(name=None, index=False))
    )
timeseries 

[SaveManager] Enqueueing save request for NVDA on <trade.assets.helpers.DataManagers_new.BulkOptionQueryRequestParameter object at 0x1443adb90>
2025-04-25 15:11:18 dbase.DataAPI.ThetaData CRITICAL: `datetime_col_name` not provided for multi index resample, setting to `Datetime`


In [240]:
timeseries.post_processed_data

Open    High    Low  Close  Midpoint  Volume  \
Optiontick       Datetime                                                     
NVDA20240315C395 2023-06-02    0.00    0.00   0.00   0.00     0.000     0.0   
                 2023-06-05   75.55   75.55  75.55  75.55    74.425     1.0   
                 2023-06-06   71.30   72.02  71.30  72.02    71.325     2.0   
                 2023-06-07   69.00   69.89  65.65  65.65    67.275   141.0   
                 2023-06-08   65.80   65.80  60.94  60.94    59.775    19.0   
...                             ...     ...    ...    ...       ...     ...   
NVDA20240315C830 2024-03-11  123.43  144.30  54.00  58.67    59.075   933.0   
                 2024-03-12   49.20   67.55  37.05  39.00    40.050  2069.0   
                 2024-03-13   57.95   90.50  42.06  90.50    90.475   688.0   
                 2024-03-14   79.57   85.80  60.00  80.10    80.425   217.0   
                 2024-03-15   66.97   76.05  41.00  50.15    50.400   704.0   

                             Openinterest  
Optiontick       Datetime                  
NVDA20240315C395 2023-06-02           0.0  
                 2023-06-05          62.0  
                 2023-06-06          62.0  
                 2023-06-07          63.0  
                 2023-06-08         149.0  
...                                   ...  
NVDA20240315C830 2024-03-11        1982.0  
                 2024-03-12        1910.0  
                 2024-03-13        2345.0  
                 2024-03-14        2370.0  
                 2024-03-15        2354.0  

[12978 rows x 7 columns]